In [1]:
import pandas as pd
from pathlib import Path
import csv
from tqdm import tqdm
from gerrychain import Graph, Partition
import json
import geopandas as gpd
import numpy as np
from pcompress import Replay
import matplotlib.pyplot as plt
import networkx as nx


DUAL_GRAPH_DIR = "dual_graphs"
STATE_SPECS_DIR = "state_specifications"
CHAIN_DIR = "raw_chains"
STATS_DIR = "ensemble_stats"

In [2]:
directory_name = "../Michigan_gingles/gingles_scores/100000"
directory = Path(directory_name)
plan_types =["state_house","state_senate"]

In [3]:
df_dict = {"state_house":{"index": [], "max_score":[], "max_score_plan_number":[]},
        "state_senate":{"index": [], "max_score":[], "max_score_plan_number":[]}}

for j, filepath in enumerate(directory.iterdir()):
    if j%500 == 0:
        print("step", j)
    if filepath.is_file():
        file_name = filepath.name
        plan_type, method = file_name.split("steps_")
        plan_type = plan_type.split("_0.05")[0].split("michigan_")[-1]
        method = method.split(".csv")[0]
        df_dict[plan_type]["index"].append(method)
        
        with open(filepath, mode='r') as f:
            csv_reader = csv.reader(f)
            
            for i, row in enumerate(csv_reader):
                if i == 1:
                    max_score, max_index = row
                    df_dict[plan_type]["max_score"].append(float(max_score))
                    df_dict[plan_type]["max_score_plan_number"].append(int(max_index))
    
    

step 0
step 500
step 1000
step 1500
step 2000
step 2500


In [4]:
dfs = {plan_type:pd.DataFrame.from_dict(df_dict[plan_type]) for plan_type in ["state_house","state_senate"]}

In [7]:
dfs["state_house"].head()

,index,max_score,max_score_plan_number
0,cw_0.0_csw_0.0_opt_burst_10,14.474667,99995
1,cw_0.0_csw_0.0_opt_burst_1000_tilt10.0,12.441559,99001
2,cw_0.0_csw_0.0_opt_burst_1000_tilt100.0,12.441559,99001
3,cw_0.0_csw_0.0_opt_burst_1000_tilt1000.0,12.441559,99001
4,cw_0.0_csw_0.0_opt_burst_1000_tilt500.0,12.441559,99001


In [5]:
dfs["state_house"].sort_values(by = "max_score", ascending = False, inplace = True)
dfs["state_house"].head(10)

,index,max_score,max_score_plan_number
67,cw_0.0_csw_0.25_opt_burst_10_tilt0.001,16.364637,99999
380,cw_0.25_csw_0.25_opt_tilt_0.0001,16.302748,99999
865,cw_0.75_csw_0.0_opt_burst_6,16.279006,99995
356,cw_0.25_csw_0.25_opt_burst_14_tilt0.0001,15.466393,99987
341,cw_0.25_csw_0.25_opt_burst_10_tilt0.0001,15.462252,99999
351,cw_0.25_csw_0.25_opt_burst_12_tilt0.0001,15.462252,99995
875,cw_0.75_csw_0.0_opt_tilt_0.0001,15.449999,99999
851,cw_0.75_csw_0.0_opt_burst_14_tilt0.0001,15.447243,99987
846,cw_0.75_csw_0.0_opt_burst_12_tilt0.0001,15.447243,99995
836,cw_0.75_csw_0.0_opt_burst_10_tilt0.0001,15.447243,99999


In [6]:
dfs["state_senate"].sort_values(by = "max_score", ascending = False, inplace = True)
dfs["state_senate"].head(10)

,index,max_score,max_score_plan_number
77,cw_0.0_csw_0.25_opt_burst_12_tilt0.001,6.361732,99995
67,cw_0.0_csw_0.25_opt_burst_10_tilt0.001,6.360922,99999
299,cw_0.25_csw_0.0_opt_burst_12_tilt0.1,6.329784,99995
652,cw_0.5_csw_0.25_opt_burst_8_tilt0.001,6.327946,99999
651,cw_0.5_csw_0.25_opt_burst_8_tilt0.0001,6.318877,99999
646,cw_0.5_csw_0.25_opt_burst_6_tilt0.0001,6.282853,99995
647,cw_0.5_csw_0.25_opt_burst_6_tilt0.001,5.499646,99995
616,cw_0.5_csw_0.25_opt_burst_10_tilt0.0001,5.497350,99999
627,cw_0.5_csw_0.25_opt_burst_12_tilt0.001,5.495416,99995
80,cw_0.0_csw_0.25_opt_burst_14,5.491043,99974


# Computing Cut Edges

In [16]:
state="Michigan"
index =99999
chain_file = "../Michigan_gingles/raw_chains/100000/michigan_state_house_0.05_bal_100000_steps_cw_0.0_csw_0.25_opt_burst_10_tilt0.001.chain"
with open("{}/{}.json".format("../"+STATE_SPECS_DIR, state)) as fin:
    state_specification = json.load(fin)

dual_graph_file = f"../dual_graphs/{state_specification['dual_graph'][plan_type]}"
graph = Graph.from_json(dual_graph_file)


plan_generator = Replay(graph, chain_file)

for i,plan in enumerate(plan_generator):
    if i==index:
        print(len(plan['cut_edges']))
        dass=[("GEOID20","assignment")]+[(graph.nodes[n]["GEOID20"],d) for n,d in plan.assignment.items()]
        import csv
        
        file_path = '../Michigan/proposed_plans/vtd_level/state_house/Gingles.csv'

        # Writing to CSV
        with open(file_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerows(dass)



/cluster/tufts/hpc/tools/anaconda/202111/lib/python3.9/subprocess.py:1052: ResourceWarning: subprocess 8735 is still running
  _warn("subprocess %s is still running" % self.pid,
/tmp/ipykernel_65649/2657758855.py:11: ResourceWarning: unclosed file <_io.BufferedReader name=58>
  plan_generator = Replay(graph, chain_file)


2491


In [18]:

index =99995
chain_file="../Michigan_gingles/raw_chains/100000/michigan_state_senate_0.05_bal_100000_steps_cw_0.0_csw_0.25_opt_burst_12_tilt0.001.chain"
with open("{}/{}.json".format("../"+STATE_SPECS_DIR, state)) as fin:
    state_specification = json.load(fin)

dual_graph_file = f"../dual_graphs/{state_specification['dual_graph'][plan_type]}"
graph = Graph.from_json(dual_graph_file)

plan_generator = Replay(graph, chain_file)

for i,plan in enumerate(plan_generator):
    if i==index:
        print(len(plan['cut_edges']))
        dass=[("GEOID20","assignment")]+[(graph.nodes[n]["GEOID20"],d) for n,d in plan.assignment.items()]
        
        file_path = '../Michigan/proposed_plans/vtd_level/state_senate/Gingles.csv'

        # Writing to CSV
        with open(file_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerows(dass)


/tmp/ipykernel_65649/770701627.py:9: ResourceWarning: unclosed file <_io.BufferedReader name=58>
  plan_generator = Replay(graph, chain_file)


1571


# Plotting high scores

In [16]:
state_vtds = gpd.read_file(f"../shapefiles/vtds/mi_pl2020_vtd.shp")    

In [8]:
with open("{}/{}.json".format("../state_specifications", "Michigan")) as fin:
        state_specification = json.load(fin)
dual_graph_file = f"../dual_graphs/{state_specification['dual_graph']['state_house']}"
graph = Graph.from_json(dual_graph_file)

new_index = {}
for node,data in graph.nodes(data= True):
    new_index[data["GEOID20"]] = node
    
graph_index = []

for geo_id in state_vtds["GEOID20"]:
    graph_index.append(new_index.get(geo_id, None))

state_vtds["graph_index"] = graph_index
state_vtds = state_vtds.dropna(subset=['graph_index'])
state_vtds.set_index("graph_index", inplace =True)
state_vtds.set_index(state_vtds.index.astype(int), inplace =True)

In [11]:
for plan_type in plan_types:
    with open("{}/{}.json".format("../state_specifications", "Michigan")) as fin:
        state_specification = json.load(fin)
    dual_graph_file = f"../dual_graphs/{state_specification['dual_graph'][plan_type]}"
    graph = Graph.from_json(dual_graph_file)

    for row in range(1):
        file_post, score, index = dfs[plan_type].iloc[row]
        
        file_name = f"../Michigan_gingles/raw_chains/100000/michigan_{plan_type}_0.05_bal_100000_steps_{file_post}.chain"
        plan_generator = Replay(graph, file_name)

        for i, partition in enumerate(plan_generator):
            if i == index:
                partition.plot(geometries=state_vtds, cmap="tab20")
                plt.title(f"{plan_type.capitalize()} Gingles Optimized")
                # plt.show()
                plt.axis('off')
                plt.tight_layout()
                plt.savefig(f"figures/{plan_type}_gingles_optimized.pdf")
                plt.clf()
                break



/cluster/tufts/hpc/tools/anaconda/202111/lib/python3.9/subprocess.py:1052: ResourceWarning: subprocess 26310 is still running
  _warn("subprocess %s is still running" % self.pid,
/tmp/ipykernel_22948/779784601.py:11: ResourceWarning: unclosed file <_io.BufferedReader name=59>
  plan_generator = Replay(graph, file_name)
/cluster/tufts/hpc/tools/anaconda/202111/lib/python3.9/subprocess.py:1052: ResourceWarning: subprocess 26880 is still running
  _warn("subprocess %s is still running" % self.pid,
/tmp/ipykernel_22948/779784601.py:11: ResourceWarning: unclosed file <_io.BufferedReader name=61>
  plan_generator = Replay(graph, file_name)


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>